# Предобработка данных для рекомендательной системы

In [1]:
import pandas as pd


class Constants:
    USER_ID = "user_id"
    ITEM_ID = "item_id"
    TIMESTAMP = "time"

    TRANSACTIONS_PATH = "/Users/alfa/Documents/diplom/graphnn-recommendation-system/data/transactions_train.csv"
    RESULT_PATH = "/Users/alfa/Documents/diplom/graphnn-recommendation-system/data/processed_transactions_train.csv"

In [2]:
def prepare_filtered_dataset(
    transactions: pd.DataFrame,
    num_customers: int = 100,
    num_articles: int = 100,
    min_articles_per_user: int = 5,
    min_users_per_article: int = 5,
) -> pd.DataFrame:
    """
    Отбираем сбалансированный набор данных:
    - num_customers: количество пользователей для отбора
    - min_articles_per_user: минимальное количество покупок у пользователя
    - min_users_per_article: минимальное количество покупателей у товара
    """

    # Фильтрация товаров
    article_popularity = transactions[Constants.ITEM_ID].value_counts()
    popular_articles = article_popularity[article_popularity >= min_users_per_article]

    # Отбираем топ-N самых активных пользователей
    selected_articles = popular_articles.head(num_articles).index

    # Окончательный набор данных
    filtered_data = transactions[
        transactions[Constants.ITEM_ID].isin(selected_articles)
    ]

    # Первичная фильтрация пользователей
    user_purchase_counts = filtered_data[Constants.USER_ID].value_counts()
    active_users = user_purchase_counts[user_purchase_counts >= min_articles_per_user]

    # Отбираем топ-N самых активных пользователей
    selected_users = active_users.head(num_customers).index

    # Фильтруем транзакции только по выбранным пользователям
    final_data = filtered_data[filtered_data[Constants.USER_ID].isin(selected_users)]

    print(f"Исходный размер данных: {len(transactions)} транзакций")
    print(f"Отобрано пользователей: {len(selected_users)}")
    print(f"Отобрано товаров: {len(selected_articles)}")
    print(f"Финальный размер данных: {len(final_data)} транзакций")

    return final_data.reset_index(drop=True)

In [3]:
transactions = pd.read_csv(Constants.TRANSACTIONS_PATH)
transactions = transactions.rename(
    columns={
        "t_dat": Constants.TIMESTAMP,
        "customer_id": Constants.USER_ID,
        "article_id": Constants.ITEM_ID,
    }
)

In [4]:
filtered_transactions = prepare_filtered_dataset(transactions=transactions)
# filtered_transactions = prepare_balanced_dataset(transactions=transactions)
filtered_transactions.to_parquet(Constants.RESULT_PATH, index=False)

Исходный размер данных: 31788324 транзакций
Отобрано пользователей: 100
Отобрано товаров: 100
Финальный размер данных: 5542 транзакций


In [5]:
transactions.head()

,time,user_id,item_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [7]:
filtered_transactions["item_id"].unique()

array([156231001, 158340001, 539723005, 562245050, 562245001, 372860002,
       372860001, 399256001, 636323001, 436261001, 111593001, 507909001,
       399223029, 554450001, 568601006, 573716012, 399223001, 537116001,
       111586001, 673677002, 228257001, 608776002, 562245018, 160442007,
       564786001, 610776002, 160442010, 484398001, 108775015, 179123001,
       568597006, 573085028, 536139006, 565379001, 554598001, 572797001,
       562245046, 678942001, 464297007, 565379002, 611415001, 706016001,
       706016002, 579541001, 695632002, 719655001, 689109001, 600886001,
       706016015, 706016003, 699080001, 351484002, 610776001, 695632001,
       706016006, 448509001, 685816002, 399256005, 673396002, 723529001,
       716672001, 572797002, 759871002, 723469001, 685816001, 711053003,
       599580017, 759871001, 741356002, 749699002, 749699001, 507910001,
       762846001, 688537011, 733749001, 730683001, 720125001, 783346001,
       554479001, 768912001, 688537004, 806388001, 